# Segmenting and Clustering Neighborhoods in Toronto

## Assignment 1

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Scrape the List of postal codes of Canada

In [2]:
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(website_url,'lxml')

table_rows=soup.find('table',{'class':'wikitable sortable'}).find_all('tr')

data=[]
cols=[]

for rows in table_rows:
    data_row=[]
    
    row_headings=rows.find_all('th')
    for col in range(len(row_headings)):
        cols.append(row_headings[col].text.strip())

    row_list=rows.find_all('td')
    for cell in row_list:
        data_row.append(cell.text.strip())
   
    data.append(data_row)
# excluding heading row to data frame

df_table=pd.DataFrame(data[1:],columns=cols)


Only process the cells that have an assigned Borough

In [3]:
df_assigned=df_table[df_table['Borough'] !='Not assigned']
df_assigned

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Combine two rows into one row with the neighborhoods separated with a comma per requirement

In [6]:
temp_df=df_assigned.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [7]:
df_merge = pd.merge(df_assigned, temp_df, on='Postal Code')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Print the number of rows of dataframe

In [8]:
df_merge.shape

(103, 3)